# Actividad 4: Selección del mejor modelo de ML

Durante la etapa de entrenamiento nos podemos encontrar con diferentes estrategias de cara a mejorar el desempeño del modelo. Una de estas estrategias es conocida como **Model Selection**, en la cual, probamos con diferentes algoritmos de Machine Learning y al final, escogemos el mejor modelo con base al que obtenga las mejores métricas. Para esta actividad, seguiremos con el mismo caso de uso descrito en la Actividad 3, que es la clasificación de cancer de mama entre los tipos **Maligno y Benigno**.

En la pasada actividad usamos SVM como clasificador, en esta Activdad 3, evaluaremos el desempeño del modelo al usar 3 clasificadores diferentes, que son: **LightGBM, MLPClassifier (perceptrón multicapa) y Logistic Regression**. Al final, tomaremos el mejor modelo con base en el mejor **Accuracy**

# Imports 

Librerias que se necesitan para el desarrollo de la actividad

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import lightgbm as ltb
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression

from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# Configurando pandas para visualización de 100 columnas

In [ ]:
pd.options.display.max_columns = 100

# Lectura del dataset

In [ ]:
path = 'https://raw.githubusercontent.com/eyberthrojas/Actividad-4-ML-/main/is_cancer.csv'
data = pd.read_csv(path, on_bad_lines='skip')

# Entrenamiento del modelo

Lo descrito en esta sección es reutilizado de la Actividad anterior como base para la evaluación de diferentes clasificadores.

In [ ]:
features_finales = [
    "radius_mean", "texture_mean", "perimeter_mean", "area_mean", "smoothness_mean",
    "compactness_mean", "concavity_mean", "concave points_mean", "symmetry_mean", 
    "fractal_dimension_mean", "diagnosis"
]

In [ ]:
# Seleccionamos solo las features de interés del dataset completo
dataset_final = data[features_finales]

In [ ]:
# Variable categórica creada a partir de la feature fractal_dimension_mean. 
dataset_final["calc_fractal_dimension_mean"] = np.where(
    1, dataset_final["fractal_dimension_mean"] < 0.055, 
    0
)

In [ ]:
# Eliminamos la variable númerica que usamos para crear la variable categórica
dataset_final.drop(['fractal_dimension_mean'], axis=1, inplace=True)

In [ ]:
# Clase que realiza el fit y transform a una variable
class ColumnSelector(BaseEstimator, TransformerMixin):
    """Select only specified columns."""
    def __init__(self, columns):
        self.columns = columns
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X[self.columns]

In [ ]:
# Features numéricas
numerical = [
    "radius_mean", "texture_mean", "perimeter_mean", "area_mean", "smoothness_mean",
    "compactness_mean", "concavity_mean", "concave points_mean", "symmetry_mean"
]

In [ ]:
# ColumnSelector: selecciona las columnas númericas del dataset de entrenamiento
# SimpleImputer: los valores faltantes los completa con la mediana de todos los valores de la feature
num_pipe = Pipeline([
    ('selector', ColumnSelector(numerical)),
    ('imputer', SimpleImputer(strategy='median'))
])

In [ ]:
# Features categóricas
categorical = ["calc_fractal_dimension_mean"]

In [ ]:
# ColumnSelector: selecciona las columnas categóricas del dataset de entrenamiento
# SimpleImputer: los valores faltantes los completa con un valor fijo
# OneHotEncoder: codificación de variables categóricas
cat_pipe = Pipeline([
    ('selector', ColumnSelector(categorical)),
    ('imputer', SimpleImputer(strategy='constant')),
    ('encoder', OneHotEncoder(handle_unknown='ignore', sparse=False))
])

In [ ]:
# Feature union de todas las features categóricas y numéricas
preprocessor = FeatureUnion([
    ('cat', cat_pipe),
    ('num', num_pipe)
])

In [ ]:
# Se elimina la variable target de los datos que contienen las features 
X = dataset_final.drop(['diagnosis'], axis = 1)

In [ ]:
# Se genera en una variable separada el target del problema, en este caso "Diagnosis"
y = dataset_final['diagnosis']

In [ ]:
# De todo el conjunto de datos se toman crean datos de train y test
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 0)
print("Size of training set:", X_train.shape)
print("Size of test set:", X_test.shape)

# Evaluación de LightGBM

In [ ]:
# Instalación LightGBM
!pip install lightgbm

In [ ]:
# Pipeline 
pipe_lightgbm = Pipeline([
    ('preprocessor', preprocessor),
    ('model', ltb.LGBMClassifier())
])

In [ ]:
# Entrenamiento
model_lightgbm = pipe_lightgbm.fit(X_train,y_train)

In [ ]:
# Predicción sobre los datos de test
y_pred = model_lightgbm.predict(X_test)

In [ ]:
# Classification report de resultados al usar LightGBM
print(classification_report(y_true=y_test, y_pred=y_pred))

*Se puede observar que en un primera iteración, al usar LightGBM se obtiene un mejor accuracy respecto a lo obtenido en la actividad anterior usando SVM. A partir de esto, ya queda a criterio del científico de datos hacer uso de búsqueda de hiperaparámetros. Este paso tambien depende si la métrica obtenida es igual o mayor a la superior. Para esta actividad, tomaremos el accuracy mas alto*

# Evaluación de MLPClassifier

In [ ]:
# Pipeline
pipe_mlp = Pipeline([
    ('preprocessor', preprocessor),
    ('model', MLPClassifier())
])

In [ ]:
# Entrenamiento
model_mlp = pipe_mlp.fit(X_train,y_train)

In [ ]:
# Predicción sobre los datos de test
y_pred = model_mlp.predict(X_test)

In [ ]:
# Classification report de resultados al usar MLPClassifier
print(classification_report(y_true=y_test, y_pred=y_pred))

Se puede observar que se obtiene un accuracy del 0.91, la cual es menor que la obtenida por LightGBM e igual a la obtenida en la actividad anterior. Haremos una optimización de hiperaparámetros para intentar aumentar esta métrica

In [ ]:
# Rango de hiperparámetros
mlp_gs = MLPClassifier(max_iter=100)
parameter_space = {
    'hidden_layer_sizes': [(10,30,10),(20,)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive']
}

In [ ]:
# Búsqueda de los mejores hiperparámetros
clf = GridSearchCV(mlp_gs, parameter_space, n_jobs=-1, cv=5)
clf.fit(X_train, y_train)

In [ ]:
# Hipaparámetros encontrados por GirdSearch para MLPClassifier
clf.best_params_

In [ ]:
# Instanciamento del modelo con los hiperparámetros encontrados
clf = MLPClassifier(**clf.best_params_)

In [ ]:
# Pipeline con nuevos hiperparámetros
pipe_mlp = Pipeline([
    ('preprocessor', preprocessor),
    ('model', clf)
])

In [ ]:
# Entrenamiento
model_mlp = pipe_mlp.fit(X_train,y_train)

In [ ]:
# Predicción sobre los datos de test
y_pred = model_mlp.predict(X_test)

In [ ]:
# Métricas con hiperparámetros encontrados
print(classification_report(y_true=y_test, y_pred=y_pred))

Podemos observar que en términos de accuracy, se obtiene un valor menor a los obtenidos en iteraciones anteriores.

# Evaluación de Logistic Regression

In [ ]:
# Pipeline final del proceso de entrenamiento
pipe_lr = Pipeline([
    ('preprocessor', preprocessor),
    ('model', LogisticRegression())
])

In [ ]:
# Entrenamiento
model_lr = pipe_lr.fit(X_train,y_train)

In [ ]:
# Predicción sobre los datos de test
y_pred = model_lr.predict(X_test)

In [ ]:
# Classification report de resultados al usar LogisticRegression
print(classification_report(y_true=y_test, y_pred=y_pred))

También, se puede observar que se obtiene un accuracy del 0.91, la cual es menor que la obtenida por LightGBM y igual a la obtenida en la actividad anterior. Nuevamente, haremos una optimización de hiperaparámetros para intentar aumentar esta métrica.

In [ ]:
# Rango de hiperparámetros
parameters = {
    "C":np.logspace(-3,3,7), 
    "penalty":["l2"], 
    "solver":['liblinear','newton-cg']
}

In [ ]:
# Búsqueda de los mejores hiperparámetros
clf = GridSearchCV(LogisticRegression(), param_grid = parameters, cv = 5, verbose=True, n_jobs=-1)
clf.fit(X_train, y_train)

In [ ]:
# Hiperparámetros encontrados
clf.best_params_

{'C': 1000.0, 'penalty': 'l2', 'solver': 'newton-cg'}

In [ ]:
# Pipeline con nuevos hiperparametros
pipe_lr = Pipeline([
    ('preprocessor', preprocessor),
    ('model', LogisticRegression(**clf.best_params_))
])

In [ ]:
# Entrenamiento
model_lr = pipe_lr.fit(X_train,y_train)

In [ ]:
# Predicción sobre los datos de test
y_pred = model_lr.predict(X_test)

In [ ]:
# Métricas con nuevos hiperparametros
print(classification_report(y_true=y_test, y_pred=y_pred))

Se puede observar, que para LogisticRegression se obtiene un mayor accuracy, 0.94. Es un accuracy alto, sin embargo, es menor al obtenido por LightGBM. 